In [1]:
import yt_dlp

def download_video(url, output_path="input_video.mp4"):
    options = {'format': 'best', 'outtmpl': output_path}
    with yt_dlp.YoutubeDL(options) as ydl:
        ydl.download([url])
    print(f"Video downloaded: {output_path}")


# link = input("Enter a video link: ")
download_video("https://www.youtube.com/watch?v=shfNmYroLhw")

[youtube] Extracting URL: https://www.youtube.com/watch?v=shfNmYroLhw
[youtube] shfNmYroLhw: Downloading webpage
[youtube] shfNmYroLhw: Downloading tv client config
[youtube] shfNmYroLhw: Downloading player 7d1d50a6
[youtube] shfNmYroLhw: Downloading tv player API JSON
[youtube] shfNmYroLhw: Downloading ios player API JSON
[youtube] shfNmYroLhw: Downloading m3u8 information
[info] shfNmYroLhw: Downloading 1 format(s): 18
[download] input_video.mp4 has already been downloaded
[download] 100% of  128.52MiB
Video downloaded: input_video.mp4


In [2]:
def get_user_inputs():
    print("🎯 Welcome to the AI Video-to-Ad Converter!")
    
    # Collect user inputs
    keywords = input("Enter keywords to guide highlight selection (comma-separated): ").strip().split(",")
    ad_length = int(input("Enter desired ad length in seconds (e.g., 15, 30): "))
    cta_text = input("Enter Call-to-Action (CTA) for the ad (e.g., 'Buy Now!'): ")

    return keywords, ad_length, cta_text

# Example usage
keywords, ad_length, cta_text = get_user_inputs()
print(f"\n🔍 Keywords: {keywords}\n⏳ Ad Length: {ad_length}s\n📢 CTA: {cta_text}")


🎯 Welcome to the AI Video-to-Ad Converter!


ValueError: invalid literal for int() with base 10: ''

In [3]:
from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector

def detect_scenes(video_path):
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector())
    video_manager.start()
    scene_manager.detect_scenes(frame_source=video_manager)
    scenes = scene_manager.get_scene_list()
    print(f"Detected {len(scenes)} scenes")
    return scenes

# Example usage
scenes = detect_scenes("input_video.mp4")


c:\Coding\anaconda3\envs\Video\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
VideoManager is deprecated and will be removed.


Detected 430 scenes


In [4]:
import torch
import clip
import cv2
from PIL import Image

# Ensure correct device (use GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Load the CLIP model and tokenizer
model, preprocess = clip.load("ViT-B/32", device=device)

cpu


In [5]:
def rank_scenes(video_path, scenes, keywords):
    scene_scores = []
    for i, (start, end) in enumerate(scenes):
        # Convert FrameTimecode to integers (frame numbers)
        start_frame = start.get_frames()
        end_frame = end.get_frames()

        # Capture the middle frame of each scene
        cap = cv2.VideoCapture(video_path)
        mid_frame = (start_frame + end_frame) // 2
        cap.set(cv2.CAP_PROP_POS_FRAMES, mid_frame)
        ret, frame = cap.read()
        cap.release()

        if not ret:
            continue

        # Preprocess the frame and compute similarity with keywords
        image = preprocess(Image.fromarray(frame)).unsqueeze(0).to(device)
        text_inputs = clip.tokenize(keywords).to(device)

        with torch.no_grad():
            image_features = model.encode_image(image)
            text_features = model.encode_text(text_inputs)
            similarity = (image_features @ text_features.T).softmax(dim=-1)

        scene_scores.append((i, similarity.max().item()))

    # Sort scenes by relevance (highest similarity first)
    scene_scores.sort(key=lambda x: x[1], reverse=True)
    return scene_scores

# Example usage
keywords = ['Boss', 'Sword', 'Battle', 'Armor']
ranked_scenes = rank_scenes("input_video.mp4", scenes, keywords)
print(ranked_scenes[:5])  # Top 5 most relevant scenes

[(123, 0.9922409057617188), (136, 0.9815152883529663), (303, 0.9760878086090088), (300, 0.9732826352119446), (15, 0.9709332585334778)]


In [6]:
import whisper

def transcribe_video(video_path):
    model = whisper.load_model("base")
    result = model.transcribe(video_path)
    print("Transcript: ", result['text'])
    return result['text']

# Example usage
transcribe_video("input_video.mp4")

c:\Coding\anaconda3\envs\Video\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcript:   Every time I dial the feet of boss my character will change to a random enemy from Elden Ring I'm gonna try and beat the game and see how difficult or easier it is Alright, um, we're here and we started as a blive pretty good I don't know what he can do, but we actually have a good chance to be able to defeat a boss so I can swing the blade and then Oh Okay, all right, we're moving out boys. Let's start with one of these Okay, oh dude, we are washing him. This is crazy dude, we're washing him Oh, there we go. Okay, no, we're cooking boys. I'm kind of worried that like we'll beat the first boss And then we'll transform into like the worst character in this man I still feel like there's gonna be some bosses that are gonna give us some trouble just because I don't know how it's gonna go All right boys It's almost time. Uh something tells me we don't need to summon anyone in for this right? Okay, what you got, bro? Let me let me start off with one of these Bunk Ah, this is fi

" Every time I dial the feet of boss my character will change to a random enemy from Elden Ring I'm gonna try and beat the game and see how difficult or easier it is Alright, um, we're here and we started as a blive pretty good I don't know what he can do, but we actually have a good chance to be able to defeat a boss so I can swing the blade and then Oh Okay, all right, we're moving out boys. Let's start with one of these Okay, oh dude, we are washing him. This is crazy dude, we're washing him Oh, there we go. Okay, no, we're cooking boys. I'm kind of worried that like we'll beat the first boss And then we'll transform into like the worst character in this man I still feel like there's gonna be some bosses that are gonna give us some trouble just because I don't know how it's gonna go All right boys It's almost time. Uh something tells me we don't need to summon anyone in for this right? Okay, what you got, bro? Let me let me start off with one of these Bunk Ah, this is fine. Wait, le

In [7]:
import subprocess

def trim_video(input_path, output_path, start_time, duration):
    command = [
        'ffmpeg',
        '-i', input_path,
        '-ss', str(start_time),
        '-t', str(duration),
        '-c:v', 'libx264',
        '-c:a', 'aac',
        output_path
    ]
    subprocess.run(command)
    print(f"Trimmed video saved to: {output_path}")

# Example usage: Trim a 15-second highlight
trim_video("input_video.mp4", "highlight.mp4", 30, 15)


Trimmed video saved to: highlight.mp4


In [10]:
def add_cta_overlay(input_path, output_path, text="Buy Now!"):
    command = [
        'ffmpeg',
        '-i', input_path,
        '-vf', f"drawtext=text='{text}':x=10:y=10:fontsize=24:fontcolor=white",
        output_path
    ]
    subprocess.run(command)
    print(f"CTA added: {output_path}")

# Example usage
add_cta_overlay("highlight.mp4", "final_ad.mp4", "Limited Time Offer!")

CTA added: final_ad.mp4
